# TruBot testing

In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

sys.path.append(str(Path().cwd().parent.parent.resolve()))

In [2]:
from trulens_eval.keys import *
from trulens_eval.slackbot import get_or_make_chain, get_answer
from trulens_eval.util import TP
from trulens_eval import Tru
from trulens_eval.tru_feedback import Huggingface
# Tru().reset_database()

/opt/homebrew/Caskroom/miniconda/base/envs/py38_trulens/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KEY SET: OPENAI_API_KEY
KEY SET: PINECONE_API_KEY
KEY SET: PINECONE_ENV
KEY SET: HUGGINGFACE_API_KEY
KEY SET: SLACK_TOKEN
KEY SET: SLACK_SIGNING_SECRET
KEY SET: COHERE_API_KEY


huggingface api: 0requests [00:00, ?requests/s]

In [3]:
proc = Tru().start_dashboard(force=True, _dev=Path.cwd().parent.parent)
thread = Tru().start_evaluator(restart=True)

Force stopping dashboard ...
killing psutil.Process(pid=42338, name='python3.8', status='running', started='16:48:46')
Starting dashboard ...
Looking for things to do. Stop me with `tru.stop_evaluator()`.

Finishing 0 task(s) done.


In [ ]:
selectors = [0,1,3,4]
messages = ["Who is Shayak?", "Wer ist Shayak?", "Kim jest Shayak?", "¿Quién es Shayak?", "Was ist QII?", "Co jest QII?"]

# selectors = selectors[0:1]
# messages = messages[0:1]

def test_bot(selector, question):
    print(selector, question)
    chain = get_or_make_chain(cid=question + str(selector), selector=selector)
    answer = get_answer(chain=chain, question=question)
    return answer

results = []

for s in selectors:
    for m in messages:
        results.append(TP().promise(test_bot, selector=s, question=m))

TP().finish()

Finishing 24 task(s) 0 Who is Shayak?
'Starting a new conversation with 0/default.'
0 Wer ist Shayak?
'Starting a new conversation with 0/default.'
0 Kim jest Shayak?
'Starting a new conversation with 0/default.'
0 ¿Quién es Shayak?
'Starting a new conversation with 0/default.'
0 Was ist QII?
'Starting a new conversation with 0/default.'
0 Co jest QII?
'Starting a new conversation with 0/default.'
1 Who is Shayak?
'Starting a new conversation with 1/lang_prompt.'
1 Wer ist Shayak?
'Starting a new conversation with 1/lang_prompt.'
1 Kim jest Shayak?
'Starting a new conversation with 1/lang_prompt.'
1 ¿Quién es Shayak?
'Starting a new conversation with 1/lang_prompt.'
1 Was ist QII?
'Starting a new conversation with 1/lang_prompt.'
1 Co jest QII?
'Starting a new conversation with 1/lang_prompt.'
✅ chain 0/default -> default.sqlite
3 Who is Shayak?
'Starting a new conversation with 3/filtered_context.'
3 Wer ist Shayak?
'Starting a new conversation with 3/filtered_context.'
✅ chain 0/defa

KEY SET: OPENAI_API_KEY
KEY SET: PINECONE_API_KEY
KEY SET: PINECONE_ENV
KEY SET: HUGGINGFACE_API_KEY
KEY SET: SLACK_TOKEN
KEY SET: SLACK_SIGNING_SECRET
KEY SET: COHERE_API_KEY


Looking for things to do. Stop me with `tru.stop_evaluator()`.Finishing 29 task(s) 

✅ record record_hash_908fa5bb68fb9ed85a140ed443d0a088 from 1/lang_prompt -> default.sqlite
⚡ feedback feedback_result_hash_f46be563f4052288484cfad9ec3e77d9 on record_hash_908fa5bb68fb9ed85a140ed443d0a088 -> default.sqlite
⚡ feedback feedback_result_hash_e74265315e4d4fc3462bb6cb8cb199af on record_hash_908fa5bb68fb9ed85a140ed443d0a088 -> default.sqlite
⚡ feedback feedback_result_hash_f87dc4652e6acaffa9258982661f2417 on record_hash_908fa5bb68fb9ed85a140ed443d0a088 -> default.sqlite


✅ record record_hash_c62aceb141ce01945614bbc0de1e79f6 from 0/default -> default.sqlite
⚡ feedback feedback_result_hash_5772182e8816e072a928e6c4db573262 on record_hash_c62aceb141ce01945614bbc0de1e79f6 -> default.sqlite
⚡ feedback feedback_result_hash_65a5e794bfc84a742e7fb268985a139d on record_hash_c62aceb141ce01945614bbc0de1e79f6 -> default.sqlite
⚡ feedback feedback_result_hash_d799714457c5ef6e17a8180296d5afb8 on record_hash_c62aceb141ce01945614bbc0de1e79f6 -> default.sqlite
!.

...

⚡ feedback feedback_result_hash_db39e4502444085a80659c6e4d503396 on record_hash_680b76e607bf61f9fe3e09e8021b5caa -> default.sqlite
.....

⚡ feedback feedback_result_hash_da90a8e81495ba6a90ca6e28b16a8354 on record_hash_5a56eb892ad16d929c94476fb9412cc5 -> default.sqlite
..........

⚡ feedback feedback_result_hash_117537b8a5fc028bb9a16fba118ffe37 on record_hash_83baf767e0a68f99ab92f39a239ee370 -> default.sqlite
.....

⚡ feedback feedback_result_hash_991d69f70c9218179c543a4279c8c48f on record_hash_489691b27d5a479239ec729d46c2d426 -> default.sqlite
.............................................................some tasks timed out.


⚡ feedback feedback_result_hash_06f09964bc9ab645b27f67782bd34d2d on record_hash_a0047ea47cd4f423ba999a515f30e2a1 -> default.sqlite
Looking for things to do. Stop me with `tru.stop_evaluator()`.Finishing 6 task(s) ......done.
Looking for things to do. Stop me with `tru.stop_evaluator()`.Finishing 0 task(s) done.
Looking for things to do. Stop me with `tru.stop_evaluator()`.Finishing 0 task(s) done.
Looking for things to do. Stop me with `tru.stop_evaluator()`.Finishing 0 task(s) done.


# Local Model Testing

In [ ]:
from langchain import ConversationChain, HuggingFacePipeline
from transformers import LlamaTokenizer, LlamaForCausalLM, pipeline

tokenizer = LlamaTokenizer.from_pretrained("chavinlo/alpaca-native")

base_model = LlamaForCausalLM.from_pretrained("chavinlo/alpaca-native", device_map='auto')

# Convert the model's parameters to a higher precision data type
base_model = base_model.float()

pipe = pipeline(
    "text-generation",
    model=base_model, 
    tokenizer=tokenizer, 
    max_length=256,
    temperature=0.6,
    top_p=0.95,
    repetition_penalty=1.2
)

local_llm = HuggingFacePipeline(pipeline=pipe)

# We are going to set the memory to go back 4 turns
# window_memory = ConversationBufferWindowMemory(k=4)

In [ ]:
conversation = ConversationChain(
    llm=local_llm, 
    verbose=True
)

In [ ]:
# Setup some feedback functions.

from trulens_eval import tru_feedback
from trulens_eval.tru_db import Record
import numpy as np

hugs = tru_feedback.Huggingface()
openai = tru_feedback.OpenAI()

# Toxicity (of the response):
f_toxic = tru_feedback.Feedback(hugs.not_toxic).on_response()

# Language match (between prompt and response):
f_lang_match = tru_feedback.Feedback(hugs.language_match).on(
    text1="prompt", text2="response"
)

# Question to answer relevance:
f_relevance = tru_feedback.Feedback(openai.relevance).on(
    prompt="input", response="output"
)

# Question to context piece relevance:
#f_qs_relevance = tru_feedback.Feedback(openai.qs_relevance).on(
#    question="input",
#    statement=Record.chain.combine_docs_chain._call.args.inputs.input_documents
#).on_multiple(
#    multiarg="statement", each_query=Record.page_content, agg=np.min
#)

feedbacks=[
    # f_toxic, 
    f_lang_match, 
    # f_relevance, 
    # f_qs_relevance
]

In [ ]:
tc = Tru().Chain(conversation, chain_id="llama-7b-hf-convo", feedbacks=feedbacks)

In [ ]:
res, record = tc.call_with_record("What is the meaning of life?")

In [ ]:
res, record = tc.call_with_record("Who is Nora?")